<a href="https://colab.research.google.com/github/richardborde/Journey_Sorter/blob/main/JourneySorter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Journey Sorter API

This notebook contains an API for sorting a list of boarding cards for various types of transport.
It will sort the cards in such a way that they form a complete journey from start to end.

### How to Run

1. Execute the cell that installs required libraries.
2. Run the cell that imports libraries.
3. Execute the cells to define the `BoardingCard` and `JourneySorter` classes.
4. Run the cell that reads data from Google Sheets.
5. Execute the sorting algorithm and see the sorted journey.

### Assumptions

- There is always an unbroken chain between all legs of the trip.
- The sorting algorithm has the lowest possible time complexity to sort the cards.

## Google Sheets Integration

This Colab notebook works with boarding card data stored in a Google Sheet. You can access the live version of this data [here](https://docs.google.com/spreadsheets/d/1WdhXWghzw692Tk2Vncw1oqc7iDgvCANixABYASB4X5w/edit?pli=1#gid=0).

Follow the instructions in the notebook to directly connect to the Google Sheet for the boarding card data.


In [19]:
# Import libraries and authenticate the user
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default

# Initialize Google Sheets API Client
creds, _ = default()
gc = gspread.authorize(creds)


In [20]:
class BoardingCard:
    """
    This class represents a boarding card for various types of transport.

    Attributes:
    - transport_type: A string representing the type of transport (e.g., 'Train', 'Bus', etc.)
    - from_city: A string representing the departure city.
    - to_city: A string representing the destination city.
    - seat: A string representing the seat assignment.
    - gate: A string representing the gate number.
    - baggage: A string representing the baggage claim or drop-off information.
    """

    def __init__(self, transport_type, departure, destination, seat, gate, baggage):
        """
        Initializes a new BoardingCard object.

        Args:
        - transport_type (str): The type of transport.
        - from_city (str): The departure city.
        - to_city (str): The destination city.
        - seat (str): The seat assignment.
        - gate (str): The gate number.
        - baggage (str): Baggage claim or drop-off information.

        Returns:
        None
        """
        self.transport_type = transport_type
        self.departure = departure
        self.destination = destination
        self.seat = seat
        self.gate = gate
        self.baggage = baggage


In [21]:
class JourneySorter:
    """
    This class takes a list of boarding cards and sorts them to form a complete journey.
    """

    def __init__(self, boarding_cards):
        """
        Initializes a new JourneySorter object with a list of unsorted boarding cards.
        """
        self.boarding_cards = boarding_cards
        self.sorted_journey = []

    def sort_journey(self):
        """
        Sorts the list of boarding cards to form a complete journey.
        """
        from_to_map = {card.departure: card for card in self.boarding_cards}
        to_from_map = {card.destination: card for card in self.boarding_cards}

        # Find the starting city
        start = None
        for city in from_to_map.keys():
            if city not in to_from_map:
                start = city
                break

        # Sort the journey
        while start:
            card = from_to_map.get(start, None)
            if card:
                self.sorted_journey.append(card)
                start = card.destination
            else:
                break


In [22]:
# Read data from the Google Sheet
worksheet = gc.open_by_key('1WdhXWghzw692Tk2Vncw1oqc7iDgvCANixABYASB4X5w').sheet1
rows = worksheet.get_all_values()

# Convert the rows to BoardingCard objects, taking into account the headers
boarding_cards = [BoardingCard(*row) for row in rows[1:]]  # Exclude header


In [23]:
print("Loaded Boarding Cards:")
for card in boarding_cards:
    print(card.__dict__)


Loaded Boarding Cards:
{'transport_type': 'bus 10', 'departure': 'Boston', 'destination': 'New York', 'seat': '3A', 'gate': '--', 'baggage': '--'}
{'transport_type': 'flight AA500', 'departure': 'Tokyo', 'destination': 'San Francisco', 'seat': '12C', 'gate': '25A', 'baggage': '150'}
{'transport_type': 'train 56B', 'departure': 'San Francisco', 'destination': 'Boston', 'seat': '7B', 'gate': '--', 'baggage': '--'}
{'transport_type': 'flight NH789', 'departure': 'Seoul', 'destination': 'Tokyo', 'seat': '4A', 'gate': '8B', 'baggage': '200'}


In [24]:
# Debug print to check if the boarding cards are correctly populated
print("Check initial boarding cards:")
for card in boarding_cards:
    print(f"Transport Type: {card.transport_type}, Departure: {card.departure}, Destination: {card.destination}, Seat: {card.seat}, Gate: {card.gate}, Baggage: {card.baggage}")


Check initial boarding cards:
Transport Type: bus 10, Departure: Boston, Destination: New York, Seat: 3A, Gate: --, Baggage: --
Transport Type: flight AA500, Departure: Tokyo, Destination: San Francisco, Seat: 12C, Gate: 25A, Baggage: 150
Transport Type: train 56B, Departure: San Francisco, Destination: Boston, Seat: 7B, Gate: --, Baggage: --
Transport Type: flight NH789, Departure: Seoul, Destination: Tokyo, Seat: 4A, Gate: 8B, Baggage: 200


In [25]:
# JourneySorter class definition
class JourneySorter:
    def __init__(self, boarding_cards):
        self.boarding_cards = boarding_cards
        self.sorted_journey = []

    def sort_journey(self):
        # Create mappings
        from_to_map = {card.departure: card for card in self.boarding_cards}
        to_from_map = {card.destination: card for card in self.boarding_cards}

        # Debug prints
        print("From-To Map:", from_to_map)
        print("To-From Map:", to_from_map)

        # Find the starting city
        for city in from_to_map.keys():
          if city not in to_from_map:
            start = city
            break

        # Sort the journey
        current = start
        while current in from_to_map:
            print(f"Current city: {current}")  # Debug line
            self.sorted_journey.append(from_to_map[current])
            current = from_to_map[current].destination
            print(f"Sorted Journey: {[card.destination for card in self.sorted_journey]}")  # Debug line

# Initialize the JourneySorter class and sort the boarding cards
journey_sorter = JourneySorter(boarding_cards)
journey_sorter.sort_journey()

# Display the sorted journey
for i, card in enumerate(journey_sorter.sorted_journey):
    print(f"{i+1}. Take {card.transport_type} from {card.departure} to {card.destination}. "
          f"Sit in seat {card.seat}. Gate: {card.gate}. Baggage: {card.baggage}")


From-To Map: {'Boston': <__main__.BoardingCard object at 0x7e0ab2d1fdf0>, 'Tokyo': <__main__.BoardingCard object at 0x7e0a965c9f00>, 'San Francisco': <__main__.BoardingCard object at 0x7e0a965ca230>, 'Seoul': <__main__.BoardingCard object at 0x7e0a965cbb80>}
To-From Map: {'New York': <__main__.BoardingCard object at 0x7e0ab2d1fdf0>, 'San Francisco': <__main__.BoardingCard object at 0x7e0a965c9f00>, 'Boston': <__main__.BoardingCard object at 0x7e0a965ca230>, 'Tokyo': <__main__.BoardingCard object at 0x7e0a965cbb80>}
Current city: Seoul
Sorted Journey: ['Tokyo']
Current city: Tokyo
Sorted Journey: ['Tokyo', 'San Francisco']
Current city: San Francisco
Sorted Journey: ['Tokyo', 'San Francisco', 'Boston']
Current city: Boston
Sorted Journey: ['Tokyo', 'San Francisco', 'Boston', 'New York']
1. Take flight NH789 from Seoul to Tokyo. Sit in seat 4A. Gate: 8B. Baggage: 200
2. Take flight AA500 from Tokyo to San Francisco. Sit in seat 12C. Gate: 25A. Baggage: 150
3. Take train 56B from San Fran